# Transforming and joining raw data

The "raw" data is divided among the following tables:

- **Customer metadata**
  - customerID
  - gender
  - date of birth (we'll derive age and senior citizen status from this)
  - Partner
  - Dependents
  - (nominal) MonthlyCharges
- **Billing events**
  - customerID
  - date (we'll derive tenure from the number/duration of billing events)
  - kind (one of "AccountCreation", "Charge", or "AccountTermination")
  - value (either a positive nonzero amount or 0.00; we'll derive TotalCharges from the sum of amounts and Churn from the existence of an AccountTermination event)
- **Customer phone features**
  - customerID
  - feature (one of "PhoneService" or "MultipleLines")
- **Customer internet features**
  - customerID
  - feature (one of "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies")
  - value (one of "Fiber", "DSL", "Yes", "No")
- **Customer account features**
  - customerID
  - feature (one of "Contract", "PaperlessBilling", "PaymentMethod")
  - value (one of "Month-to-month", "One year", "Two year", "No", "Yes", "Credit card (automatic)", "Mailed check", "Bank transfer (automatic)", "Electronic check")

We want to join these together to reconstitute a training data set with this schema:

- customerID
- gender
- SeniorCitizen
- Partner
- Dependents
- tenure
- PhoneService
- MultipleLines
- InternetService
- OnlineSecurity
- OnlineBackup
- DeviceProtection
- TechSupport
- StreamingTV
- StreamingMovies
- Contract
- PaperlessBilling
- PaymentMethod
- MonthlyCharges
- TotalCharges
- Churn

In [1]:
# notebook parameters

import os

spark_master = "local[*]"
app_name = "churn-etl"
input_files = dict(
    billing="billing_events", 
    account_features="customer_account_features", 
    internet_features="customer_internet_features", 
    meta="customer_meta", 
    phone_features="customer_phone_features"
)
output_file = "churn-etl"
output_prefix = ""
output_mode = "overwrite"
output_kind = "csv"
input_kind = "csv"


In [2]:
import pyspark

session = pyspark.sql.SparkSession.builder \
    .master(spark_master) \
    .appName(app_name) \
    .config("spark.eventLog.enabled", True) \
    .config("spark.eventLog.dir", ".") \
    .getOrCreate()
session

In [3]:
def read_df(fn):
    kwargs = {}
    if input_kind == "csv":
        kwargs['header'] = True
    return getattr(session.read, "input_kind")("%s.%s" % (fn, input_kind), **kwargs)

# Reconstructing billing events and charges

In [4]:
billing_events = read_df(input_files["billing"])
billing_events.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: string (nullable = true)



In [5]:
billing_events = billing_events.withColumn("value", billing_events.value.cast("float"))

In [6]:
import pyspark.sql.functions as F

counts_and_charges = billing_events. \
   groupBy("customerID", "kind"). \
   agg(
    F.count(billing_events.value).alias("event_counts"), 
    F.sum(billing_events.value).alias("total_charges")
   )


In [7]:
customers = billing_events.select("customerID").distinct()

terminations = billing_events.where(
    F.col("kind") == "AccountTermination"
).select(
    F.col("customerID").alias("Churn")
)

churned = customers.join(
    terminations, 
    customers.customerID == terminations.Churn, 
    how="leftouter"
).select(
    "customerID", 
    F.when(
        F.col("Churn").isNull(), "No"
    ).otherwise("Yes").alias("Churn")
)

customer_charges = customers.join(
        counts_and_charges.where(F.col("kind") == "Charge"), 
        "customerID"   
    ).select(
        "customerID",
        F.col("event_counts").alias("tenure"),
        F.col("total_charges").alias("TotalCharges")
    )
    
customer_billing = churned.join(
    customer_charges, 
    "customerID"
)

In [8]:
customer_billing.show()

+----------+-----+------+------------------+
|customerID|Churn|tenure|      TotalCharges|
+----------+-----+------+------------------+
|0117-LFRMW|  Yes|    37|1448.8000221252441|
|0356-ERHVT|   No|    11| 521.8999900817871|
|0786-VSSUD|  Yes|    16| 679.8499755859375|
|0953-LGOVU|   No|    12|432.24998474121094|
|1450-SKCVI|  Yes|    56|  4092.85009765625|
|1894-IGFSG|  Yes|    22|1907.8499298095703|
|2011-TRQYE|  Yes|    18| 599.9999771118164|
|2592-YKDIF|   No|     1|20.350000381469727|
|2712-SYWAY|   No|     1|25.700000762939453|
|3195-TQDZX|   No|     3| 61.45000076293945|
|3528-HFRIQ|   No|    71|3765.0500144958496|
|3623-FQBOX|   No|    21|416.39999771118164|
|3668-QPYBK|  Yes|     2| 108.1500015258789|
|3692-JHONH|   No|    52| 5621.850158691406|
|4248-QPAVC|  Yes|    17|1463.4500427246094|
|4868-AADLV|   No|    66|           7862.25|
|5668-MEISB|   No|    72| 7657.399841308594|
|5802-ADBRC|   No|    50| 5232.899856567383|
|5859-HZYLF|   No|    26| 515.7499961853027|
|6234-RAAP

# Reconstructing phone features


In [9]:
phone_features = read_df(input_files["phone_features"])
phone_features.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- feature: string (nullable = true)
 |-- value: string (nullable = true)



In [10]:
phone_service = phone_features.where(
    F.col("feature") == "PhoneService"
).select("customerID", F.lit("Yes").alias("PhoneService"))

multiple_lines = phone_features.where(
    F.col("feature") == "MultipleLines"
).select("customerID", F.lit("Yes").alias("MultipleLines"))


In [11]:
customer_phone_features = customers.join(
    phone_service,
    "customerID", 
    how="leftouter"
).join(
    multiple_lines,
    "customerID", 
    how="leftouter"    
).select(
    "customerID",
    F.when(
        F.col("PhoneService").isNull(), "No"
    ).otherwise("Yes").alias("PhoneService"),
    "MultipleLines"
).select(
    "customerID",
    "PhoneService",
    F.when(
        F.col("PhoneService") == "No", "No phone service"
    ).otherwise(
        F.when(
            F.col("MultipleLines").isNull(), "No"
        ).otherwise("Yes")
    ).alias("MultipleLines")
)

# Reconstructing internet features

Whereas phone features only include whether or not there are multiple lines, there are several internet-specific features in accounts:

- `InternetService` (one of `Fiber optic` or `DSL` in the "raw" data; its absence translates to `No` in the processed data)
- `OnlineSecurity` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)
- `OnlineBackup` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)
- `DeviceProtection` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)
- `TechSupport` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)
- `StreamingTV` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)
- `StreamingMovies` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)

This will lead to some slightly more interesting joins!

In [12]:
internet_features = read_df(input_files["internet_features"])
internet_features.printSchema()
internet_features.show()

root
 |-- customerID: string (nullable = true)
 |-- feature: string (nullable = true)
 |-- value: string (nullable = true)

+----------+----------------+-----------+
|customerID|         feature|      value|
+----------+----------------+-----------+
|9412-GHEEC| InternetService|Fiber optic|
|9412-GHEEC|DeviceProtection|        Yes|
|9412-GHEEC|     TechSupport|        Yes|
|9412-GHEEC|     StreamingTV|        Yes|
|9412-GHEEC| StreamingMovies|        Yes|
|9415-DPEWS| InternetService|Fiber optic|
|9415-DPEWS|     TechSupport|        Yes|
|9415-DPEWS|     StreamingTV|        Yes|
|9415-TPKRV| InternetService|        DSL|
|9415-TPKRV|    OnlineBackup|        Yes|
|9415-ZNBSX| InternetService|        DSL|
|9415-ZNBSX|  OnlineSecurity|        Yes|
|9415-ZNBSX|    OnlineBackup|        Yes|
|9415-ZNBSX|DeviceProtection|        Yes|
|9415-ZNBSX|     TechSupport|        Yes|
|9415-ZNBSX|     StreamingTV|        Yes|
|9415-ZNBSX| StreamingMovies|        Yes|
|9419-IPPBE| InternetService|Fiber o

In [13]:
def untidy_feature(df, feature):
    """ 'untidies' a feature by turning it into a column """
    return df.where(
        F.col("feature") == feature
    ).select("customerID", F.col("value").alias(feature))

internet_service = untidy_feature(internet_features, "InternetService")

online_security = untidy_feature(internet_features, "OnlineSecurity")

online_backup = untidy_feature(internet_features, "OnlineBackup")

device_protection = untidy_feature(internet_features, "DeviceProtection")

tech_support = untidy_feature(internet_features, "TechSupport")

streaming_tv = untidy_feature(internet_features, "StreamingTV")

streaming_movies = untidy_feature(internet_features, "StreamingMovies")


In [14]:
def chained_join(column, base_df, dfs, how="leftouter"):
    acc = base_df
    for df in dfs:
        acc = acc.join(df, column, how=how)

    return acc

customer_internet_features = chained_join(
    "customerID", 
    customers,
    [
        internet_service, 
        online_security, 
        online_backup, 
        device_protection, 
        tech_support, 
        streaming_tv, 
        streaming_movies
    ]
)


In [15]:
def resolve_nullable_column(df, col, null_val="No"):
    return F.when(
        df[col].isNull(), null_val
    ).otherwise(
        df[col]
    ).alias(col)

def resolve_dependent_column(df, col, parent_col="InternetService", 
                             null_val="No", 
                             null_parent_val="No internet service"):
    return F.when(
        df[parent_col] == "No", null_parent_val
    ).otherwise(
        F.when(
            df[col].isNull(), null_val
        ).otherwise(df[col])
    ).alias(col)

customer_internet_features = customer_internet_features.select(
    "customerID",
    resolve_nullable_column(customer_internet_features, "InternetService"),
    resolve_dependent_column(customer_internet_features, "OnlineSecurity", "InternetService"),
    resolve_dependent_column(customer_internet_features, "OnlineBackup", "InternetService"),
    resolve_dependent_column(customer_internet_features, "DeviceProtection", "InternetService"),
    resolve_dependent_column(customer_internet_features, "TechSupport", "InternetService"),
    resolve_dependent_column(customer_internet_features, "StreamingTV", "InternetService"),
    resolve_dependent_column(customer_internet_features, "StreamingMovies", "InternetService")
)

# Reconstructing account features

In [16]:
account_features = read_df(input_files["account_features"])
account_features.printSchema()
account_features.show()

root
 |-- customerID: string (nullable = true)
 |-- feature: string (nullable = true)
 |-- value: string (nullable = true)

+----------+-------------+--------------------+
|customerID|      feature|               value|
+----------+-------------+--------------------+
|7590-VHVEG|PaymentMethod|    Electronic check|
|5575-GNVDE|PaymentMethod|        Mailed check|
|3668-QPYBK|PaymentMethod|        Mailed check|
|7795-CFOCW|PaymentMethod|Bank transfer (au...|
|9237-HQITU|PaymentMethod|    Electronic check|
|9305-CDSKC|PaymentMethod|    Electronic check|
|1452-KIOVK|PaymentMethod|Credit card (auto...|
|6713-OKOMC|PaymentMethod|        Mailed check|
|7892-POOKP|PaymentMethod|    Electronic check|
|6388-TABGU|PaymentMethod|Bank transfer (au...|
|9763-GRSKD|PaymentMethod|        Mailed check|
|7469-LKBCI|PaymentMethod|Credit card (auto...|
|8091-TTVAX|PaymentMethod|Credit card (auto...|
|0280-XJGEX|PaymentMethod|Bank transfer (au...|
|5129-JLPIS|PaymentMethod|    Electronic check|
|3655-SNQYZ|

In [17]:
contracts = untidy_feature(account_features, "Contract")

paperless = untidy_feature(account_features, "PaperlessBilling")

payment = untidy_feature(account_features, "PaymentMethod")

customer_account_features = chained_join(
    "customerID", 
    customers,
    [contracts, paperless, payment]
)

customer_account_features = customer_account_features.select(
    "customerID",
    "Contract",
    resolve_nullable_column(customer_account_features, "PaperlessBilling"),
    "PaymentMethod"
)

# Account metadata

In [18]:
account_meta = read_df(input_files["meta"])

account_meta.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- dateOfBirth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- MonthlyCharges: string (nullable = true)
 |-- now: string (nullable = true)



In [19]:
account_meta.select(
    "customerID",
    F.when(
        F.col("now") >= F.add_months(F.col("dateOfBirth"), 65 * 12), 
        "Yes"
    ).otherwise("No").alias("SeniorCitizen"),
    "Partner",
    "Dependents",
    "gender",
    "MonthlyCharges"
).show()

+----------+-------------+-------+----------+------+--------------+
|customerID|SeniorCitizen|Partner|Dependents|gender|MonthlyCharges|
+----------+-------------+-------+----------+------+--------------+
|5774-QPLTF|           No|    Yes|       Yes|  Male|         20.35|
|5774-XZTQC|           No|    Yes|       Yes|Female|         20.45|
|5777-KJIRB|           No|     No|        No|Female|         50.25|
|5777-ZPQNC|           No|     No|        No|Female|         20.15|
|5778-BVOFB|           No|     No|        No|Female|          59.5|
|5780-INQIK|           No|     No|        No|Female|          49.4|
|5781-BKHOP|           No|    Yes|        No|Female|        100.65|
|5781-RFZRP|           No|    Yes|        No|  Male|          73.5|
|5787-KXGIY|           No|    Yes|        No|  Male|          19.3|
|5788-YPOEG|           No|    Yes|       Yes|Female|         84.75|
|5789-LDFXO|           No|     No|        No|  Male|          24.6|
|5791-KAJFD|           No|    Yes|       Yes|Fem

# Putting it all together

In [20]:
wide_data = chained_join(
    "customerID",
    customers,
    [
        customer_billing,
        customer_phone_features,
        customer_internet_features,
        customer_account_features,
        account_meta
    ]
).select(
    "customerID", 
    "gender", 
    "SeniorCitizen", 
    "Partner", 
    "Dependents", 
    "tenure", 
    "PhoneService", 
    "MultipleLines", 
    "InternetService", 
    "OnlineSecurity", 
    "OnlineBackup", 
    "DeviceProtection", 
    "TechSupport", 
    "StreamingTV", 
    "StreamingMovies", 
    "Contract", 
    "PaperlessBilling", 
    "PaymentMethod", 
    "MonthlyCharges", 
    "TotalCharges", 
    "Churn"
)

In [21]:
wide_data.explain()

== Physical Plan ==
*(34) Project [customerID#16, gender#334, SeniorCitizen#335, Partner#336, Dependents#337, tenure#65L, PhoneService#138, MultipleLines#142, InternetService#240, OnlineSecurity#241, OnlineBackup#242, DeviceProtection#243, TechSupport#244, StreamingTV#245, StreamingMovies#246, Contract#293, PaperlessBilling#311, PaymentMethod#299, MonthlyCharges#338, TotalCharges#66, Churn#50]
+- *(34) BroadcastHashJoin [customerID#16], [customerID#332], LeftOuter, BuildRight
   :- *(34) Project [customerID#16, Churn#50, tenure#65L, TotalCharges#66, PhoneService#138, MultipleLines#142, InternetService#240, OnlineSecurity#241, OnlineBackup#242, DeviceProtection#243, TechSupport#244, StreamingTV#245, StreamingMovies#246, Contract#293, PaperlessBilling#311, PaymentMethod#299]
   :  +- SortMergeJoin [customerID#16], [customerID#422], LeftOuter
   :     :- *(27) Project [customerID#16, Churn#50, tenure#65L, TotalCharges#66, PhoneService#138, MultipleLines#142, InternetService#240, OnlineSec

In [22]:
def write_df(df, name):
    name = "%s.%s" % (name, output_kind)
    if output_prefix != "":
        name = "%s-%s" % (output_prefix, name)
    kwargs = {}
    if output_kind == "csv":
        kwargs['header'] = True
    getattr(df.write.mode(output_mode), output_kind)(name, **kwargs)

In [23]:
write_df(wide_data, output_file)

In [24]:
session.stop()